# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

Project Goal: Determine which customers are most likely to stop buying service so company can assign them an account manager. 

# Configuration

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Churn').getOrCreate()

In [ ]:
spark.conf.set('spark.sql.repl.eagerEval.enabled',True)
spark.conf.set('spark.sql.repl.eagerEval.maxNumRows',5)

In [ ]:
import pyspark.sql.functions as F

# Data 

In [ ]:
data = spark.read.csv('customer_churn.csv',inferSchema=True,header=True)

In [ ]:
data

Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
Cameron Williams,42.0,11066.8,0,7.22,8.0,2013-08-30 07:00:40,10265 Elizabeth M...,Harvey LLC,1
Kevin Mueller,41.0,11916.22,0,6.5,11.0,2013-08-13 00:38:46,6157 Frank Garden...,Wilson PLC,1
Eric Lozano,38.0,12884.75,0,6.67,12.0,2016-06-29 06:20:07,1331 Keith Court ...,"Miller, Johnson a...",1
Phillip White,42.0,8010.76,0,6.71,10.0,2014-04-22 12:43:12,13120 Daniel Moun...,Smith Inc,1
Cynthia Norton,37.0,9191.58,0,5.56,9.0,2016-01-19 15:31:15,765 Tricia Row Ka...,Love-Jones,1


In [ ]:
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [ ]:
data.groupBy(["Account_Manager","Churn"]).count()

Account_Manager,Churn,count
1,0,349
1,1,84
0,0,401
0,1,66


In [ ]:
print(f"% of people with account managers who churned {84/349} \n without {66/401}")

% of people with account managers who churned 0.24068767908309455 
 without 0.16458852867830423


In [ ]:
data.select(F.corr("Account_Manager","Churn"))

"corr(Account_Manager, Churn)"
0.07061077173214911


An aside, it appears that assigning account does increase the probability of a customer churning. This statement assumes the account managers were actually randomly assigned. 

In [ ]:
#Missing Data


data.select(*(F.sum(F.col(c).isNull().cast('int')).alias(c) for c in data.columns))

Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company,Churn
0,0,0,0,0,0,0,0,0,0


# Data Transformation

---



In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [ ]:
# Create features vector and label 

inputs = ['Age','Total_Purchase','Years','Num_Sites']
assembler = VectorAssembler(inputCols=inputs, outputCol='features')

In [ ]:
final_data = assembler.transform(data).select("features",F.col("Churn").alias('label'))
final_data

features,label
"[42.0,11066.8,7.2...",1
"[41.0,11916.22,6....",1
"[38.0,12884.75,6....",1
"[42.0,8010.76,6.7...",1
"[37.0,9191.58,5.5...",1


In [ ]:
train, test = final_data.randomSplit([0.7,0.3])

# Model Fitting

In [ ]:
# Model 

from pyspark.ml.classification import LogisticRegression
logr_obj = LogisticRegression()

In [ ]:
model = logr_obj.fit(train)

In [ ]:
# Evaluate training

training_summary = model.summary

In [ ]:
training_summary.predictions.describe().filter("summary = 'mean'")

summary,label,prediction
mean,0.1573373676248109,0.11043872919818457


In [ ]:
training_summary.accuracy

0.9016641452344932

In [ ]:
# Evalate on test data

test_pred = model.transform(test)

In [ ]:
test_pred

features,label,rawPrediction,probability,prediction
"[29.0,9617.59,5.4...",0,[4.45400096781760...,[0.98850181143016...,0.0
"[29.0,13240.01,4....",0,[7.10591875763788...,[0.99918043648415...,0.0
"[29.0,13255.05,4....",0,[4.40477539397537...,[0.98792864751762...,0.0
"[30.0,6744.87,5.1...",0,[3.54814676791932...,[0.97202708008424...,0.0
"[30.0,10744.14,7....",1,[2.21806819461296...,[0.90186034765228...,0.0


In [ ]:
# Model Evaluation

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
my_eval = BinaryClassificationEvaluator()

In [ ]:
my_eval.evaluate(results) #AUC

0.8852032520325176

# Use model to predict new data

In [ ]:
new_data = spark.read.csv('new_customers.csv',inferSchema=True,header=True)

In [ ]:
new_data

Names,Age,Total_Purchase,Account_Manager,Years,Num_Sites,Onboard_date,Location,Company
Andrew Mccall,37.0,9935.53,1,7.71,8.0,2011-08-29 18:37:54,38612 Johnny Stra...,King Ltd
Michele Wright,23.0,7526.94,1,9.28,15.0,2013-07-22 18:19:54,21083 Nicole Junc...,Cannon-Benson
Jeremy Chang,65.0,100.0,1,1.0,15.0,2006-12-11 07:48:13,085 Austin Views ...,Barron-Robertson
Megan Ferguson,32.0,6487.5,0,9.4,14.0,2016-10-28 05:32:13,922 Wright Branch...,Sexton-Golden
Taylor Young,32.0,13147.71,1,10.0,8.0,2012-03-20 00:36:46,Unit 0789 Box 073...,Wood LLC


In [ ]:
# Transformed test data with company "index"

new_data2 = assembler.transform(new_data).select("Company","features")

In [ ]:
new_pred = model.transform(new_data2)

In [ ]:
new_pred

Company,features,rawPrediction,probability,prediction
King Ltd,"[37.0,9935.53,7.7...",[2.88010479136537...,[0.94685413709500...,0.0
Cannon-Benson,"[23.0,7526.94,9.2...",[-5.7250087595295...,[0.00325271004167...,1.0
Barron-Robertson,"[65.0,100.0,1.0,1...",[-3.6581830592873...,[0.02513143854037...,1.0
Sexton-Golden,"[32.0,6487.5,9.4,...",[-4.9588561831798...,[0.00697200380919...,1.0
Wood LLC,"[32.0,13147.71,10...",[1.79865143993315...,[0.85798469647227...,0.0


# Answer Project Question

In [ ]:
p_df = new_pred.toPandas()

In [ ]:
import pandas as pd 
p_df = pd.DataFrame(p_df).loc[:,['Company','probability']]

In [ ]:
p_df

,Company,probability
0,King Ltd,"[0.9468541370950049, 0.05314586290499512]"
1,Cannon-Benson,"[0.0032527100416714183, 0.9967472899583286]"
2,Barron-Robertson,"[0.025131438540375264, 0.9748685614596247]"
3,Sexton-Golden,"[0.0069720038091936245, 0.9930279961908064]"
4,Wood LLC,"[0.8579846964722773, 0.14201530352772274]"
5,Parks-Robbins,"[0.15429621842701427, 0.8457037815729858]"


In [ ]:
f = lambda x: x[1]
p_df['Prob of Churn'] = p_df.probability.apply(f)
p_df = p_df.drop('probability',axis=1)

In [ ]:
p_df[['Company','Prob of Churn']].sort_values(by='Prob of Churn',ascending=False)

,Company,Prob of Churn
1,Cannon-Benson,0.996747
3,Sexton-Golden,0.993028
2,Barron-Robertson,0.974869
5,Parks-Robbins,0.845704
4,Wood LLC,0.142015
0,King Ltd,0.053146


Depending on the threshold, Wood LLC and King Ltd are the only companies that don't need an account manager.